In [3]:
from tape.datasets import PrositFragmentationDataset

In [4]:
from prosittransformer.utils import SequenceConverter

In [5]:
import numpy as np

In [6]:
Data = PrositFragmentationDataset("/sdd/PrositToTapeDataConverter/LMDB/", split="test")

In [7]:
from collections import OrderedDict

In [8]:
IUPAC_VOCAB = OrderedDict([
    ("<pad>", 0),
    ("<mask>", 1),
    ("<cls>", 2),
    ("<sep>", 3),
    ("<unk>", 4),
    ("A", 5),
    ("B", 6),
    ("C", 7),
    ("D", 8),
    ("E", 9),
    ("F", 10),
    ("G", 11),
    ("H", 12),
    ("I", 13),
    ("K", 14),
    ("L", 15),
    ("M", 16),
    ("N", 17),
    ("O", 18),
    ("P", 19),
    ("Q", 20),
    ("R", 21),
    ("S", 22),
    ("T", 23),
    ("U", 24),
    ("V", 25),
    ("W", 26),
    ("X", 27),
    ("Y", 28),
    ("Z", 29)])

In [9]:
int2aa = {v:k for k, v in dict(IUPAC_VOCAB).items()}

In [10]:
def Int2AA(seq):
    return "".join([int2aa[s] for s in seq])

In [11]:
peptides = list()
intensity = list()
ce = list()
charge = list()
for i in range(len(Data)):
    peptide = Int2AA(Data[i][0][1:-1])
    #Dont use M(Ox)
    if not "X" in peptide:
        #only charge 2 and 3
        c = Data[i][4]
        c = np.where(c)[0][0]
        if 0 < c and c < 3:
            peptides.append(peptide)
            intensity.append(Data[i][2])
            charge.append(c)
            ce.append(Data[i][3])


In [18]:
len(Data)

1419705

In [19]:
len(peptides)

1035031

In [12]:
%config Completer.use_jedi = False

In [74]:
import pandas as pd

In [78]:
df = pd.DataFrame([[p, col, c + 1] for p, c, col in zip(peptides, charge, ce)], 
             columns=["modified_sequence","collision_energy","precursor_charge"])

In [80]:
df.to_csv("/sdd/prosittransformer/data/prosit_input.csv", index=False, sep=",")

In [13]:
len(intensity)

1035031

In [14]:
import pickle as pkl

In [15]:
pkl.dump(intensity, open("/sdd/prosittransformer/data/intensity.pkl", "wb"))

In [17]:
peptides[-1]

'DLNEKHPNLVHKRGIYK'

In [73]:
len(peptides)

1035031

In [72]:
peptides[0]

'INIDHKFHRHL'

In [64]:
set(charge)

{0, 1, 2, 3, 4}

In [53]:
Data[i][4]

array([0, 0, 1, 0, 0, 0], dtype=uint8)

In [58]:
np.where(Data[i][4])[0][0]

2

In [57]:
c

2

In [32]:
for p in peptides:
    if "X" in p:
        print(p)
        break

XNMLAGPQPYGGSK


In [40]:
Data[i][4]

array([0, 0, 1, 0, 0, 0], dtype=uint8)

In [2]:
import h5py
filename = "/sdd/PrositToTapeDataConverter/hdf5/hcd/HDF5/prediction_hcd_ho.hdf5"
data_dict = {}
keys = ["collision_energy_aligned_normed", "precursor_charge_onehot", "sequence_integer"]
with h5py.File(filename, "r") as f:
    # List all groups
    print("Keys: %s" % f.keys())
    a_group_key = list(f.keys())
    
    for k in keys:
        ## Get the data
        data_dict[k] = list(f[k])

Keys: <KeysViewHDF5 ['collision_energy', 'collision_energy_aligned_normed', 'collision_energy_normed', 'intensities_pred', 'intensities_raw', 'masses_raw', 'method', 'precursor_charge_onehot', 'rawfile', 'reverse', 'scan_number', 'score', 'sequence_integer', 'spectral_angle']>


KeyboardInterrupt: 